## 1. Extração das regras do Manual do Guia Prático

Existe o [site](https://www.i2pdf.com/) que poderia auxiliar na extração das tabelas para csv, porém existe alguns problemas, que também encontrei quando fui atras de uma biblioteca para extrair as tabelas

Ferramentas Pesquisadas
`PyPDF2`: é usado para manipular o arquivo PDF.
`PyMuPDF`:  para extrair texto de PDFs simples.
Já o `Camelot` ou `Tabula` para extrair tabelas estruturadas, porém entre os dois o melhor que perfomou foi o `Camelot`

Proposta: Essas tabelas extraídas terá os campos dos registros e será possível no passo 2 extrair as regras de validação para cada campo. Ideal seria RAG para gerar as regras de validações que devemos considerar nos dados. 

Configuração Inicial

In [2]:
import os
import sys
import pandas as pd

In [3]:
# Adicionar caminho absoluto do diretório dos scripts
sys.path.append(os.path.abspath("../scripts"))

from extract_tables import processar_pasta

In [4]:
# Configurar caminhos absolutos
caminho_base = os.path.abspath(os.path.join(os.getcwd(), ".."))
pasta_entrada = os.path.abspath(os.path.join(caminho_base, "data", "pdfs"))  # PDFs originais
pasta_saida = os.path.abspath(os.path.join(caminho_base, "data", "processed", "registros"))  # CSVs processados

# Verificar caminhos
print(f"Caminho de entrada: {pasta_entrada}")
print(f"Caminho de saída: {pasta_saida}")

Caminho de entrada: e:\Carla Reis\Projects\Notebooks\fiscal-rag\data\pdfs
Caminho de saída: e:\Carla Reis\Projects\Notebooks\fiscal-rag\data\processed\registros


In [5]:
# Processar todos os PDFs
resultados = processar_pasta(pasta_entrada, pasta_saida)

# Mostrar resultados
print("Resultados do processamento:")
display(resultados)

c:\Users\clr_c\anaconda3\envs\fiscalrag\lib\site-packages\camelot\utils.py:740: UserWarning:   (551.64, 554.13) does not lie in column range (48.71017487395207, 551.2888847138252)
  warnings.warn(
c:\Users\clr_c\anaconda3\envs\fiscalrag\lib\site-packages\camelot\utils.py:740: UserWarning:   (551.64, 554.64) does not lie in column range (48.71017487395207, 551.2888847138252)
  warnings.warn(


Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Obrig'], dtype='object', name=0)
Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Obrig'], dtype='object', name=0)
Processado e:\Carla Reis\Projects\Notebooks\fiscal-rag\data\pdfs\0200-0205.pdf - 2 tabela(s) processada(s).
Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Obrig'], dtype='object', name=0)
Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Entr', 'Saída'], dtype='object', name=0)
Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Entr', 'Saída'], dtype='object', name=0)
Processado e:\Carla Reis\Projects\Notebooks\fiscal-rag\data\pdfs\C100.pdf - 3 tabela(s) processada(s).
Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Entr', 'Saída'], dtype='object', name=0)
Coluna Anterior:Index(['Nº', 'Campo', 'Descrição', 'Tipo', 'Tam', 'Dec', 'Entr', 'Saída'], dtype='object', name=0)
Processa

,Arquivo,Status
0,0200-0205.pdf,✅ Sucesso
1,C100.pdf,✅ Sucesso
2,C170.pdf,✅ Sucesso


In [6]:
# Diretório onde os arquivos .csv estão localizados
diretorio = "../data/processed/registros"

# Listar todos os arquivos .csv no diretório
arquivos_csv = [arquivo for arquivo in os.listdir(diretorio) if arquivo.endswith(".csv")]

# Verificar se há arquivos .csv no diretório
if not arquivos_csv:
    print("Nenhum arquivo .csv encontrado no diretório.")
else:
    # Ler e exibir cada arquivo .csv
    for arquivo in arquivos_csv:
        caminho_arquivo = os.path.join(diretorio, arquivo)
        print(f"\nConteúdo do arquivo: {arquivo}")
        
        # Ler o arquivo .csv
        df = pd.read_csv(caminho_arquivo)
        
        # Exibir as primeiras linhas da tabela
        print(df.head())  # Exibe as primeiras 5 linhas por padrão
        print("-" * 50)  # Separador visual


Conteúdo do arquivo: registro_0200.csv
   Nº         Campo                                          Descrição Tipo  \
0   1           REG                         Texto fixo contendo "0200"    C   
1   2      COD_ITEM                                     Código do item    C   
2   3    DESCR_ITEM                                  Descrição do item    C   
3   4     COD_BARRA  Representação alfanumérico do código de barrad...    C   
4   5  COD_ANT_ITEM  Código  anterior  do  item  com  relação  à  ú...    C   

   Tam Dec                 Obrig  
0  004   -                     O  
1  060   -                     O  
2    -   -                     O  
3    -   -                    OC  
4  060   -  N (informar no 0205)  
--------------------------------------------------

Conteúdo do arquivo: registro_0205.csv
   Nº           Campo                                          Descrição Tipo  \
0   1             REG                         Texto fixo contendo "0205"    C   
1   2  DESCR_ANT_ITEM 

## 2. Extração das regras com REGEX ou pelo próprio modelo de LLM 

### O QUE VALIDAR?

Campo 01 (REG)

Validação do Registro:

Valor Válido:

Validação:

Preenchimento:

Guia Prático EFD-ICMS/IPI – Versão 3.1.7  
Atualização: 14 de agosto de 2024 

IMPORTANTE: 

Observações:   

Nível hierárquico - 3
Ocorrência - 1:N (um ou vários por registro C100)
 

## 3. AI Generativa (RAG) explicando os erros com base nas regras de validação (Exemplo ludico)

In [9]:
from llama_index.core import SimpleDirectoryReader
documentos = SimpleDirectoryReader(input_files=["../data/exemplos/regras.json"]).load_data()

# Imprimir o conteúdo de cada documento
for i, documento in enumerate(documentos):
    print(f"Documento {i + 1}:")
    print(documento.text)  # Acessa o texto do documento
    print("-" * 50)  # Separador para melhor visualização

Documento 1:
{
    "0200": [
      {
        "campo": "COD_ITEM",
        "obrigatorio": true,
        "formato": "\\d+",
        "mensagem_erro": "COD_ITEM deve ser numérico e único."
      },
      {
        "campo": "COD_NCM",
        "obrigatorio": true,
        "formato": "^\\d{4}\\.\\d{2}\\.\\d{2}$",
        "mensagem_erro": "COD_NCM inválido. Formato correto: NNNN.NN.NN."
      },
      {
        "campo": "UNID_INV",
        "obrigatorio": true,
        "valores_permitidos": ["UN", "KG", "LT"],
        "mensagem_erro": "UNID_INV deve ser UN, KG ou LT."
      }
    ]
  }
--------------------------------------------------


In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import time
from tenacity import retry, wait_exponential, stop_after_attempt

# Configurar LLM (usando Mistral via Ollama)
llm = Ollama(model="llama2", request_timeout=60.0)
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Carregar contexto das regras

indice = VectorStoreIndex.from_documents(documentos, embed_model=embed_model)
motor_rag = indice.as_query_engine(llm=llm)

# Função para explicar erros
@retry(wait=wait_exponential(multiplier=1, min=4, max=10), stop=stop_after_attempt(5))
def explicar_erro(erro):
    resposta = motor_rag.query(
        f"Explique o seguinte erro de validação do EFD ICMS/IPI em português: '{erro}'. Seja claro e direto."
    )
    return resposta.response

# Exemplo de uso
erros = [
    "Linha 2: COD_NCM inválido. Formato correto: NNNN.NN.NN.",
    "Linha 3: UNID_INV deve ser UN, KG ou LT."
]

for erro in erros:
    explicacao = explicar_erro(erro)
    print(f"**Erro**: {erro}\n**Explicação RAG**: {explicacao}\n")

**Erro**: Linha 2: COD_NCM inválido. Formato correto: NNNN.NN.NN.
**Explicação RAG**:  The error message you provided indicates that the value entered for the "COD_NCM" field is invalid. According to the context information provided, the correct format for this field is "^\\d{4}\\.\\d{2:\\.\\d{2}$". In other words, the COD_NCM must be a valid VAT number in the format of "NNNN.NN.NN", where N represents a digit.

Therefore, the error message is indicating that the value entered for COD_NCM does not match this format, and it must be corrected in order to proceed with the validation process.

**Erro**: Linha 3: UNID_INV deve ser UN, KG ou LT.
**Explicação RAG**:  The error message " Linha 3: UNID_INV deve ser UN, KG ou LT." indicates that the value entered for the "UNID_INV" field is not valid. According to the context information provided, the valid values for this field are "UN", "KG", and "LT". If the value entered does not match one of these options, it will be considered invalid and 

#### Traduzido pelo Tradutor

**Erro**: Linha 2: COD_NCM inválido. Formato correto: NNNN.NN.NN.

**Explicação RAG**: A mensagem de erro que você forneceu indica que o valor inserido para o campo "COD_NCM" é inválido. De acordo com as informações de contexto fornecidas, o formato correto para este campo é "^\\d{4}\\.\\d{2:\\.\\d{2}$". Em outras palavras, o COD_NCM deve ser um número de IVA válido no formato "NNNN.NN.NN", onde N representa um dígito.

Portanto, a mensagem de erro está indicando que o valor inserido para COD_NCM não corresponde a este formato e deve ser corrigido para prosseguir com o processo de validação.

-------------------------------

**Erro**: Linha 3: UNID_INV deve ser UN, KG ou LT.

**Explicação RAG**: A mensagem de erro " Linha 3: UNID_INV deve ser UN, KG ou LT." indica que o valor inserido para o campo "UNID_INV" não é válido. De acordo com as informações de contexto fornecidas, os valores válidos para este campo são "UN", "KG" e "LT". Se o valor inserido não corresponder a uma dessas opções, ele será considerado inválido e a validação falhará. Em outras palavras, o campo requer um formato ou valor específico, e a entrada atual não atende a esse requisito.